In [ ]:
from telethon import TelegramClient
import telethon
from telethon import functions, utils
from telethon.tl.types import Message, Channel
from telethon.tl.types.messages import DialogFilters
from telethon.tl.custom.file import File
from tqdm import tqdm
from dotenv import load_dotenv
from typing import List, Dict, Any
import os
import asyncio
import json
from datetime import date, datetime
from telethon.errors import FloodWaitError
from sqlmodel import Session, create_engine, text, SQLModel, select, update, delete
from sqlalchemy.engine.base import Engine
from models.telegram import MediaItem, Channel


load_dotenv('../.env')  # take environment variables from .env.
api_id = os.environ['TG_API_ID']
api_hash = os.environ['TG_API_HASH']
phone = os.environ['TG_PHONE']

# Paths
MEDIA_PATH = '../static/media/'
DB_PATH = '../teledeck.db'
engine = create_engine(f"sqlite:///{DB_PATH}")

# Ensure media directory exists
os.makedirs(MEDIA_PATH, exist_ok=True)

class DownloadProgressBar(tqdm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs, leave=False)
    def update_to(self, current, total):
        self.total = total
        self.update(current - self.n)


"""
async def download_media(client, message: Message) -> str:
    with DownloadProgressBar(unit='B', unit_scale=True) as pb:
        return await client.download_media(message, MEDIA_PATH, progress_callback=pb.update_to)
"""
async def download_media(client, message: Message) -> str:
    return await client.download_media(message, MEDIA_PATH)


# Flood prevention
MAX_CONCURRENT_TASKS = 5
semaphore = asyncio.Semaphore(MAX_CONCURRENT_TASKS)


async def delay():
    await asyncio.sleep(.1)

async def exponential_backoff(attempt):
    wait_time = 2 ** attempt
    print(f"Rate limit hit. Waiting for {wait_time} seconds before retrying.")
    await asyncio.sleep(wait_time)

async def process_with_semaphore(callback: asyncio.Task, task_id: int) -> None:
    async with semaphore:
        max_attempts = 5
        for attempt in range(max_attempts):
            try:
                await delay()
                await callback
                break
            except FloodWaitError as e:
                if attempt < max_attempts - 1:
                    await exponential_backoff(attempt)
                else:
                    print(f"Max attempts reached. Skipping: {task_id}")
                    return



## Experiments and scratch

In [ ]:
# Init client
client = TelegramClient(username, api_id, api_hash)
await client.connect()

In [ ]:
# Demo from documentation

async def main():
    # Getting information about yourself
    me = await client.get_me()

    # "me" is a user object. You can pretty-print
    # any Telegram object with the "stringify" method:
    print(me.stringify())

    # When you print something, you see a representation of it.
    # You can access all attributes of Telegram objects with
    # the dot operator. For example, to get the username:
    username = me.username
    print(username)
    print(me.phone)

    # You can print all the dialogs/conversations that you are part of:
    # async for dialog in client.iter_dialogs():
    #     print(dialog.name, 'has ID', dialog.id)

    # You can send messages to yourself...
    await client.send_message('me', f'Hello, {username}!')
    # ...to some chat ID
    # await client.send_message(-100123456, 'Hello, group!')
    # ...to your contacts
    # await client.send_message('+34600123123', 'Hello, friend!')
    # ...or even to any username
    # await client.send_message('username', 'Testing Telethon!')

    # You can, of course, use markdown in your messages:
    message = await client.send_message(
        'me',
        'This message has **bold**, `code`, __italics__ and '
        'a [nice website](https://example.com)!',
        link_preview=False
    )

    # Sending a message returns the sent message object, which you can use
    print(message.raw_text)

    # You can reply to messages directly if you have a message object
    await message.reply('Cool!')

    # Or send files, songs, documents, albums...
    # await client.send_file('me', '/home/me/Pictures/holidays.jpg')

    # You can print the message history of any chat:
    async for message in client.iter_messages('me', limit=10):
        print(message.id, message.text)

        # You can download media from messages, too!
        # The method will return the path where the file was saved.
        """
        if message.photo:
            path = await message.download_media()
            print('File saved to', path)  # printed after download is done
        """

async with client:
    await main()

In [ ]:
# Browse all chats

dialogs = await client.get_dialogs()

dialog_by_id = {}
for d in dialogs:
    real_id, peer_type = utils.resolve_id(d.id)
    print(real_id, peer_type)
    dialog_by_id[real_id] = d

# client.get_channels(dialog_by_id.keys())

In [ ]:
# Find recent messages in a chat
filtered = filter(lambda dialog: dialog.name.find('Abrahams') > -1, dialogs)
f: telethon.tl.custom.dialog.Dialog = next(filtered)
last_messages = await client.get_messages(f, limit=20)

[print(m) for m in last_messages]

In [ ]:
# Delete most recent message
# await client.delete_messages(f, last_messages[0], revoke=True)

In [ ]:
# Find folders
chatFolders: DialogFilters = await client(functions.messages.GetDialogFiltersRequest())
# print(chatFolders.stringify())

In [ ]:
# Find chats in media folder
mediaFolder = None
for folder in chatFolders.filters:
    if not hasattr(folder, 'title'):
        continue
    if folder.title == 'MediaView':
        mediaFolder = folder
        break

# print(mediaFolder.stringify())


# target_channels = [dialog_by_id[peer.channel_id] for peer in mediaFolder.include_peers]
# target_channels = [client.get_entity(peer) for peer in mediaFolder.include_peers]
# await client.get_entity(peer)

target_channels = await asyncio.gather(*[client.get_entity(peer) for peer in mediaFolder.include_peers])
for channel in target_channels:
    print(channel.stringify())

In [ ]:
# Test messages in the first two channels
"""
for channel_id in target_channels[0:2]:
    async for message in client.iter_messages(channel_id, limit=5):
        print(message.stringify())
"""

# Grab most recent post @ self to test twitter embed
test_message = None
async for message in client.iter_messages('me', limit=1):
    test_message = message

print(test_message.stringify())

# test_message.media = MessageMediaWebPage
print(test_message.document)

In [ ]:
# Grab posts in channel

async def collect_posts(client, target_channels, limit=5):
    post_collection = []
    for channel in target_channels:
        async for message in client.iter_messages(channel, limit=limit):
            post_collection.append(message)
    return post_collection
posts = await collect_posts(client, target_channels, limit=5)

## Older version

In [ ]:


async def get_media_info(client, message: Message, channel: Channel, downloaded_files: Dict[str, str]) -> Dict[str, Any]:
    media_info = {
        'id': message.id,
        'date': str(message.date),  # Convert to string for JSON serialization
        'channel': channel.title,
        'text': message.text,  # Include message text
    }

    if message.file:
        file_id = message.file.id
        if file_id in downloaded_files:
            media_info['path'] = downloaded_files[file_id]
        else:
            media_info['path'] = await download_media(client, message)
            downloaded_files[file_id] = media_info['path']

        if message.photo:
            media_type = 'photo'
        elif message.gif:
            media_type = 'gif'
        elif message.video:
            media_type = 'video'
        elif message.document:
            mime_type = message.document.mime_type
            media_type = mime_type.split('/')[-1] if 'image/' in mime_type else 'document'
        else:
            media_type = 'unknown'

        media_info.update({
            'type': media_type,
            'file_id': file_id,
            'file_name': message.file.name,
            'file_size': message.file.size,
        })
    elif message.web_preview:
        media_info.update({
            'type': 'web_preview',
            'url': message.web_preview.url,
        })
    else:
        return None

    return media_info

async def collect_media(client, message_collection: List[Message], channels: Dict[int, Channel]) -> List[Dict[str, Any]]:
    media_collection = []
    downloaded_files = {}
    for message in tqdm(message_collection):
        channel = channels.get(message.peer_id.channel_id)
        media_info = await get_media_info(client, message, channel, downloaded_files)
        if media_info:
            media_collection.append(media_info)
            print(f"Has {media_info['type']}: {message.id}")
        else:
            print(f"No media found: {message.id}")
    return media_collection


# Export to JSON
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    elif isinstance(obj, telethon.tl.custom.file.File):
        return json.dumps({
            'id': obj.id, 'name': obj.name, 'ext': obj.ext, 'mime_type': obj.mime_type})
    elif hasattr(obj, 'stringify'):
        return obj.stringify()
    raise TypeError ("Type %s not serializable" % type(obj))

def export_to_json(media_collection: List[Dict[str, Any]], filename: str = 'media_data.json') -> None:
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(media_collection, f, ensure_ascii=False, indent=2, default=json_serial)
    print(f"Media data exported to {filename}")





### NEW CODE ###

async def process_channel(client, channel):
    download_folder = './static/media/'
    os.makedirs(download_folder, exist_ok=True)

    async for message in client.iter_messages(channel, limit=10):  # Adjust limit as needed
        if message.media:
            file_path = await download_media(client, message, download_folder)
            if file_path:
                file_name = os.path.basename(file_path)
                file_id = message.file.id if message.file else None
                media_type = message.file.mime_type.split('/')[0] if message.file else 'unknown'

                with Session(engine) as session:
                    session.add(MediaItem(
                        file_id=file_id,
                        date=message.date,
                        channel=channel.title,
                        text=message.text,
                        type=media_type,
                        file_name=file_name,
                        file_size=message.file.size if message.file else 0,
                        url=f'/media/{file_name}'
                    ))


await client.disconnect()

## New version

In [ ]:
client = TelegramClient(username, api_id, api_hash)
await client.connect()
engine = create_engine(f"sqlite:///{DB_PATH}")


In [ ]:
async def collect_posts(
    client, target_channels: List[Channel], limit: int = 10
) -> List[Message]:
    return [
        message
        for channel in target_channels
        async for message in client.iter_messages(channel, limit=limit)
    ]


async def process_message(client, message: Message, channel: Channel) -> None:
    if message.file:
        file: File = message.file
        file_id = file.media.id
        if file.size > 50_000_000:
            print(f"Skipping large file: {file_id}")
            return
        if file.sticker_set is not None:
            print(f"Skipping sticker: {file_id}")
            return

        # Check if file_id already exists in database
        with Session(engine) as session:
            statement = select(MediaItem).where(MediaItem.file_id == file_id)
            existing = session.exec(statement).first()
            if existing:
                print(f"Skipping existing file: {file_id}")
                if existing.channel_id is None:
                    existing.channel_id = channel.id
                    existing.message_id = message.id
                    existing.file_id = file_id
                    session.commit()

                print(f"Skipping download for existing file_id: {file_id}")
                return

        file_path = await download_media(client, message)
        if file_path:
            file_name = os.path.basename(file_path)

            if message.video or file_name.lower().endswith(".mp4"):
                media_type = "video"
            elif message.gif:
                media_type = "gif"
            elif message.photo:
                media_type = "photo"
            elif message.document:
                mime_type = message.document.mime_type
                media_type = (
                    mime_type.split("/")[-1] if "image/" in mime_type else "document"
                )
            else:
                media_type = "unknown"

            with Session(engine) as session:
                session.add(MediaItem(
                    file_id=file_id,
                    channel_id=channel.id,
                    message_id=message.id,
                    date=message.date,
                    text=message.text,
                    type = media_type,
                    file_name=file_name,
                    file_size = file.size,
                    url= f"/media/{file_name}",
                ))
                session.commit()
            print(f"Processed {media_type}: {file_id}")
    elif message.web_preview:
        print(f"Skipping web preview: {message.id}")
    else:
        print(f"No media found: {message.id}")


async def process_channel(client: TelegramClient, channel: Channel):
    print(f"Processing channel: {channel.title}")
    with Session(engine) as session:
        query = select(MediaItem.message_id).\
                    where(MediaItem.channel_id == channel.id).\
                    where(MediaItem.message_id != None).\
                    order_by(MediaItem.message_id.asc())
        oldest_post = session.exec(query).first()


    if oldest_post:
        message_task = client.iter_messages(channel, limit=100, offset_id = oldest_post[0])
    else:
        message_task = client.iter_messages(channel, limit=100)


    tasks = []
    async for message in message_task:
        # Adjust limit as needed - 300
        # print(f"Message is {(datetime.now().astimezone() - message.date).days} days old")
        # if (datetime.now().astimezone() - message.date).days > 7:
        #    break
        task = asyncio.create_task(
            process_with_semaphore(
                process_message(client, message, channel), message.id
            )
        )
        tasks.append(task)
    await asyncio.gather(*tasks)


# PROBLEMS
# 1. How to handle media that is not a file (e.g. web previews)?
# 2. Check what happens to Twitter embeds
# 3. Paginate / search by date?
async def main(engine: Engine):
    chat_folders: DialogFilters = await client(
        functions.messages.GetDialogFiltersRequest()
    )
    media_folder = next(
        (
            folder
            for folder in chat_folders.filters
            if hasattr(folder, "title") and folder.title == "MediaView"
        ),
        None,
    )

    if not media_folder:
        print("MediaView folder not found")
        return

    target_channels = await asyncio.gather(
        *[client.get_entity(peer) for peer in media_folder.include_peers]
    )
    # Keep only channels where channel.title is not in the database

    with Session(engine) as session:
        existing_channels = session.get(MediaItem.channel).distinct()
        target_channels = [
            channel
            for channel in target_channels
            if channel.title not in existing_channels
        ]
    print([channel.title for channel in target_channels])
    await asyncio.gather(
        *[process_channel(client, engine, channel) for channel in target_channels]
    )

    print("Processing complete")


# Run the main function
with Session(engine) as session:
    await main(engine)
await main()

In [ ]:

await client.disconnect()

# Examine individual messages

In [ ]:
client = TelegramClient(username, api_id, api_hash)
await client.connect()
engine = create_engine(f"sqlite:///{DB_PATH}")


In [ ]:

channel_title = 'GIFs and Vids'

async def find_channel() -> telethon.types.Dialog | None:
    async for dialog in client.iter_dialogs():
        d: telethon.types.Dialog = dialog
        if d.title.find(channel_title) > -1:
            print(d)
            return d
    print("Channel not found.")
    return None

target_channel = await find_channel()


In [ ]:
messages = await client.get_messages(target_channel, 2, filter=telethon.tl.types.InputMessagesFilterUrl())

In [ ]:
message = messages[0]
print(message.stringify())

In [ ]:


"""
if message.file:
    file: File = message.file
    file_id = file.media.id
    if file.size > 50_000_000:
        print(f"Skipping large file: {file_id}")
        return
    if file.sticker_set is not None:
        print(f"Skipping sticker: {file_id}")
        return

    # Check if file_id already exists in database
    with Session(engine) as session:
        statement = select(MediaItem).where(MediaItem.file_id == file_id)
        existing = session.exec(statement).first()
        if existing:
            print(f"Skipping existing file: {file_id}")
            if existing.channel_id is None:
                existing.channel_id = channel.id
                existing.message_id = message.id
                existing.file_id = file_id
                session.commit()

            print(f"Skipping download for existing file_id: {file_id}")
            return

    file_path = await download_media(client, message)
    if file_path:
        file_name = os.path.basename(file_path)

        if message.video or file_name.lower().endswith(".mp4"):
            media_type = "video"
        elif message.gif:
            media_type = "gif"
        elif message.photo:
            media_type = "photo"
        elif message.document:
            mime_type = message.document.mime_type
            media_type = (
                mime_type.split("/")[-1] if "image/" in mime_type else "document"
            )
        else:
            media_type = "unknown"

        with Session(engine) as session:
            session.add(MediaItem(
                file_id=file_id,
                channel_id=channel.id,
                message_id=message.id,
                date=message.date,
                text=message.text,
                type = media_type,
                file_name=file_name,
                file_size = file.size,
                url= f"/media/{file_name}",
            ))
            session.commit()
        print(f"Processed {media_type}: {file_id}")
elif message.web_preview:
    print(f"Skipping web preview: {message.id}")
else:
    print(f"No media found: {message.id}")

"""


In [ ]:

message
print(message.media.webpage.has_large_media) # true
web_document = message.media.webpage.document
print(web_document.mime_type == 'video/mp4') # true
dir(web_document)
# dir(message)
print(message.file.mime_type)
web_document.id
# print(message.file)
# print(dir(message.file))
print(message.file.media.id)
#print(message.video.stringify()) # 5824722488059233779

In [ ]:

dl = await client.download_media(web_document, MEDIA_PATH)

In [ ]:
dl

## Database migrations

In [ ]:




def update_table():
    with Session(engine) as session:
        session.exec(text("ALTER TABLE media_items ADD COLUMN user_deleted BOOLEAN DEFAULT false"))
        session.commit()

def fix_extension_types():
    with Session(engine) as session:
        media_items = session.exec(select(MediaItem)).all()
        updated_count = 0
        for item in media_items:
            extension = os.path.splitext(item.file_name)[1]
            if extension in ['mp4', 'webm'] and item.type != 'video':
                item.type = 'video'
                updated_count += 1
                print(f"Updated entry {item.id}: {item.file_name} from {item.type} to video")

            file_path = os.path.join(MEDIA_PATH, item.file_name)
            if os.path.exists(file_path):
                _, file_extension = os.path.splitext(file_path)
                if file_extension.lower() != '.mp4' and item.type == 'video':
                    new_file_name = f"{os.path.splitext(item.file_name)[0]}.mp4"
                    new_file_path = os.path.join(MEDIA_PATH, new_file_name)
                    os.rename(file_path, new_file_path)
                    item.file_name = new_file_name
                    item.url = f'/media/{new_file_name}'
                    updated_count += 1
                    print(f"Renamed file and updated entry {item.id}: {item.file_name} to {new_file_name}")

        session.commit()
        print(f"Updated {updated_count} entries")

def set_viewed_true():
    with Session(engine) as session:
        session.exec(update(MediaItem).values(seen=True))
        session.commit()

def fix_unavailable_entries():
    with Session(engine) as session:
        query = select(MediaItem).where(MediaItem.user_deleted == False)
        updated_count = 0
        for item in session.exec(query):
            target_file = os.path.join(MEDIA_PATH, item.file_name)
            if not os.path.exists(target_file):
                item.user_deleted = True
                updated_count += 1
                print(f"Missing entry {item.id} - file {item.file_name}")
        session.commit()
        print(f"Updated {updated_count} entries")
# fix_unavailable_entries()

def mark_favorites_from_downloads():
    with Session(engine) as session:
        static_files = {item.name: item for item in os.scandir('../static/media') if item.is_file()}
        for item in os.scandir('/mnt/e/Downloads'):
            if item.is_file() and item.name in static_files:
                media_item = session.exec(select(MediaItem).where(MediaItem.file_name == item.name)).first()
                if media_item:
                    print("Found!")
                    media_item.favorite = True
        session.commit()



## Search

In [ ]:

client = TelegramClient(username, api_id, api_hash)
await client.connect()

In [ ]:
twitter_users = json.load(open('./data/twitter_users.json'))
twitter_users

In [ ]:
from telethon.tl.types import InputMessagesFilterEmpty, InputPeerEmpty, PeerChannel
from telethon.tl.types.messages import MessagesSlice


def makeSearchRequest(query: str, limit: int = 50):
    return telethon.functions.messages.SearchGlobalRequest(
        q=query,
        filter=InputMessagesFilterEmpty(),
        min_date=-1,
        max_date=-1,
        offset_rate=0,
        offset_peer=InputPeerEmpty(),
        offset_id=-1,
        limit=limit,
    )

channel_ids = set()


In [ ]:
print(results.stringify())

In [ ]:

for twitter_user in tqdm(twitter_users):
    searchRequest = makeSearchRequest(twitter_user, 50)
    results: MessagesSlice = await client(searchRequest)
    for message in results.messages:
        if isinstance(message.peer_id, PeerChannel):
            channel_ids.add(message.peer_id.channel_id)


In [ ]:
channel_ids = sorted(list(channel_ids))
# with open('./data/searched_channels.json', 'w') as f:
    # json.dump(list(channel_ids), f, indent=2)

In [ ]:

async def id_to_channel(client, channel_id):
    channel = await client.get_entity(channel_id)
    return channel


channels: List[Channel] = await asyncio.gather(*[id_to_channel(client, channel_id) for channel_id in channel_ids])


In [ ]:
https://t.me/+Qdrj_WFxpW-Kfwrb
print(sorted([channel.username for channel in channels if channel.username is not None]))